<h1>Battle of the Neighborhoods</h1>

<p><strong>Welcome!</strong> This notebook is about comparing different neighborhoods in terms of a service, search for potential explanation of why a neighborhood is popular, the cause of complaints in another neighborhood, or anything else related to neighborhoods. So what you will learn to do is given a city like the City of Toronto, you will segment it into different neighborhoods using the geographical coordinates of the center of each neighborhood, and then using a combination of location data and machine learning, you will group the neighborhoods into clusters. You will be required to leverage location  data to solve a problem or to get deeper insights into a neighborhoods reputation. So let's get started!<p>

In [7]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!
